In [1]:
import tensorflow as tf
import logging
import pickle
import random
import os
from tensorflow.contrib import rnn
# from rouge import rouge


os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"


/data1/songbohan/anaconda3/envs/reinforce_learning/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
# add train log module
log_fn = "./log/train.20181017.log"
logger = logging.getLogger("training")
hdlr = logging.FileHandler(log_fn)
logger.addHandler(hdlr)
logger.setLevel(logging.INFO)
tensorboard_log_path = "./log/"
model_dump_dir="./ckpt/basis"
# model_load_dir="../model/20180918"
inference_fn = "./log/produce.20181017.samples"

In [ ]:
# basic_data_
data_fn = "../data/finance150.pkl"
with open(data_fn, 'rb') as f:
    word2idx, idx2word, X_train, X_test, Y_train, Y_test = pickle.load(open(data_fn, 'rb'))
# X_test, Y_test = X_test[:100], Y_test[:100]

In [ ]:
# data_fn = "../data/finance.kb.0823.pkl"
# with open(data_fn, 'rb') as f:
#     X_test, Y_test = pickle.load(f)

In [ ]:
print(len(X_test))

1000


In [ ]:
# hyper parameter

beg, eos, emp, unk = 0, 1, 2, 3
learning_rate = 0.001

save_epoc_step = 2
dropout_keep_prob = 1.
restore = False
inference = False
train = True
batch_size = 128
epocs = 1500

maxlena = 200
maxlent = 20
maxlen = maxlena + maxlent
maxlenh = maxlent
maxlend = maxlena

vocab_size = len(word2idx)
# idx2word = {idx:word for word, idx in word2idx.items()}
embedding_size = 100
memory_dim = 512

# cnn A, C
filter_sizes = [3,3,3,3,3]
num_filters = 1




In [ ]:
# building model

feed_batch_size = tf.placeholder(tf.int32, [], name="feed_batch_size")
encoder_word_inputs = tf.placeholder(tf.int32, shape=[None, maxlend], name="encoder_word_inputs")
encoder_pos_inputs = tf.placeholder(tf.int32, shape=[None, maxlend], name="encoder_pos_inputs")
decoder_targets = tf.placeholder(tf.int32, shape=[None, maxlenh], name="decoder_targets")
decoder_inputs = tf.placeholder(tf.int32, shape=[None, maxlenh], name="decoder_inputs")
writer = tf.summary.FileWriter(tensorboard_log_path, graph=tf.get_default_graph())


# user_embeddings = tf.placeholder(tf.float32, shape=[vocab_size, embedding_size], name="user_embeddings")
word_embeddings = tf.get_variable(initializer=tf.truncated_normal(shape=[vocab_size, embedding_size], stddev=0.1), name="word_embeddings")
pos_embeddings = tf.Variable(tf.random_uniform([maxlend+3, embedding_size], -1,0, 1.0), name="pos_embeddings")

# writer = tf.summary.FileWriter(tensorboard_log_path, graph=tf.get_default_graph())


encoder_word_inputs_embedded = tf.nn.embedding_lookup(word_embeddings, encoder_word_inputs)
enocder_pos_inputs_embedded = tf.nn.embedding_lookup(pos_embeddings, encoder_pos_inputs)
encoder_inputs_embedded = encoder_word_inputs_embedded + enocder_pos_inputs_embedded
embed_expanded = tf.expand_dims(encoder_inputs_embedded, -1)

def CNN_C_encoder(embed_expanded):
        
    cnn_c_inputs = embed_expanded
    for i, filter_size in enumerate(filter_sizes):
        with tf.variable_scope("conv-c-%s" % i):
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = tf.get_variable(initializer=tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.get_variable(initializer=tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(embed_expanded, W, strides=[1,1,1,1], padding="SAME", name="conv")
            cnn_c_inputs = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            print(W.name)
        
    h = tf.squeeze(cnn_c_inputs, axis=3)
#     with tf.name_scope("dropout"):
#         h_drop = tf.nn.dropout(h, dropout_keep_prob, name="dropout")
    
    return h

In [ ]:
cnn_c_outputs = CNN_C_encoder(embed_expanded)
print(cnn_c_outputs.shape)

conv-c-0/W:0
conv-c-1/W:0
conv-c-2/W:0
conv-c-3/W:0
conv-c-4/W:0
(?, 200, 100)


In [ ]:
def CNN_A_encoder(embed_expanded):
    
    cnn_a_inputs = embed_expanded
    for i, filter_size in enumerate(filter_sizes):
        with tf.variable_scope("conv-a-%s" % i):
            filter_shape = [filter_size, embedding_size, 1, num_filters]

            W = tf.get_variable(initializer=tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.get_variable(initializer=tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(embed_expanded, W, strides=[1,1,1,1], padding="SAME", name="conv")
            cnn_a_inputs = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            
            print(W.name)
        
    h = tf.squeeze(cnn_a_inputs, axis=3)
    
#     with tf.name_scope("dropout"):
#         h_drop = tf.nn.dropout(h, dropout_keep_prob, name="dropout")
    
    return h

In [ ]:
cnn_a_outouts = CNN_A_encoder(embed_expanded)
print(cnn_a_outouts.shape)

conv-a-0/W:0
conv-a-1/W:0
conv-a-2/W:0
conv-a-3/W:0
conv-a-4/W:0
(?, 200, 100)


In [ ]:


def LSTM_decoder(cnn_a_outouts, cnn_c_outputs, decoder_inputs):
    
    decoder_inputs_embedded = tf.nn.embedding_lookup(word_embeddings, decoder_inputs)
    lstm = tf.contrib.rnn.BasicLSTMCell(memory_dim)
    decoder_outputs = []
    with tf.variable_scope("attetion"):
        state = lstm.zero_state(feed_batch_size, tf.float32)
#         cell_init = tf.placeholder(tf.float32, shape=[None, memory_dim], name="init_cell")
#         hidden_init = tf.placeholder(tf.float32, shape=[None, memory_dim], name="init_hidden")
#         state = rnn.LSTMStateTuple(cell_init, hid_init)
        W = tf.get_variable(initializer=tf.truncated_normal([memory_dim, embedding_size], stddev=0.1), name="W")
        b = tf.get_variable(initializer=tf.constant(0.1, shape=[embedding_size]), name="b")
        print(W.name)
    
    decoder_outputs = []

#     cns = []
    with tf.variable_scope("RNN"):
        for i in range(maxlenh): 
            d = tf.expand_dims(tf.matmul(state[1], W) + b + decoder_inputs_embedded[:,i,:], -1)
            alpha = tf.expand_dims(tf.nn.softmax(tf.squeeze(tf.matmul(cnn_a_outouts, d), axis=-1), dim=1), axis=-1)
            context = tf.reduce_sum(tf.multiply(alpha, cnn_c_outputs), axis=1)
            current_input = tf.concat([context, decoder_inputs_embedded[:,i,:]], axis=1)

            if i > 0:
                tf.get_variable_scope().reuse_variables() 

            decoder_output, state = lstm(current_input, state)
            decoder_outputs.append(decoder_output)
        
#             cns.append(cnn_c_outputs)
    return tf.stack(decoder_outputs, axis=1)

In [ ]:
decoder_outputs =  LSTM_decoder(cnn_a_outouts, cnn_c_outputs, decoder_inputs)
print(decoder_outputs.shape)

attetion/W:0
(?, 20, 512)


In [ ]:
# writer=tf.summary.FileWriter('./improved_graph2',tf.get_default_graph())

In [ ]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocab_size)
labels = tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32)
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=decoder_logits)

In [ ]:
print(decoder_logits.name)
print(labels.shape)
print(decoder_logits.shape)

fully_connected/BiasAdd:0
(?, 20, 378486)
(?, 20, 378486)


In [ ]:
print(labels.shape)
print(stepwise_cross_entropy.shape)


(?, 20, 378486)
(?, 20)


In [ ]:
loss = tf.reduce_mean(stepwise_cross_entropy, name="loss")
tf.summary.scalar("cost", loss)
summary_op = tf.summary.merge_all()

decoder_prediction = tf.argmax(decoder_logits, 2, name="decoder_prediction")
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, name="op_adam_minize")
saver = tf.train.Saver()

In [ ]:
def rpadd(x, maxlen=maxlenh, eos=eos, prefix=None):
    
    if prefix != None:
        x = [prefix] + x
    n = len(x)
    if n > maxlen - 1:
        x = x[:maxlen - 1]
        n = maxlen - 1
    res = x + [eos] + [emp] * (maxlen - 1 - n)
    
    assert len(res) == maxlen
    
    return res

In [ ]:
def ids2sent(x):
    sent = []
    for idx in x:
#         if idx == emp:
#             continue
        if idx == eos:
            break
        sent.append(idx2word[idx])
    
    return " ".join(sent)
        

In [ ]:
def predict(X_test, Y_test, sess, log_dir):
    
    batch_size = 1
    
    rouge_mea = {
        'rouge_1/f_score': 0.0,
        'rouge_1/r_score': 0.0,
        'rouge_1/p_score': 0.0,
        'rouge_2/f_score': 0.0,
        'rouge_2/r_score': 0.0,
        'rouge_2/p_score': 0.0,
        'rouge_l/f_score': 0.0,
        'rouge_l/r_score': 0.0,
        'rouge_l/p_score': 0.0
    }
    outf = open(log_dir, 'w', encoding='utf-8')
    j = 0
    while j < len(X_test):
        
        encoder_word_inputs_ = list(map(lambda x: rpadd(x, maxlend), X_test[j:j+batch_size]))
        encoder_pos_inputs_ = list(map(lambda x:rpadd(list(range(3,len(x)+3)), maxlend), X_test[j:j+batch_size]))
        decoder_inputs_ = list(map(lambda x: rpadd(x, maxlenh, prefix=beg), Y_test[j:j+batch_size]))
        j = j + batch_size
        
      
        test_x = [[] for _ in range(len(encoder_word_inputs_))]
        for l in range(maxlenh):
            new_decoder_input_ = list(map(lambda x: rpadd(x, maxlenh, prefix=beg), test_x))

            decoder_prediction_, decoder_logits_ = sess.run([decoder_prediction, decoder_logits],
                                          feed_dict={
                                              encoder_word_inputs: encoder_word_inputs_,
                                              encoder_pos_inputs: encoder_pos_inputs_,
                                              decoder_inputs: new_decoder_input_,
                                              feed_batch_size: 1
                                          })
            for b in range(len(encoder_word_inputs_)):
                next_word_idx = decoder_prediction_[b][l]
                test_x[b].append(next_word_idx)
                        
                
        # compute 'rouge' measure in the batch
        hypotheses = [ids2sent(single) for single in test_x]
        references = [ids2sent(single) for single in decoder_inputs_]
        
        rouge_batch = rouge(hypotheses, references)
        
        for key in rouge_mea.keys():
            rouge_mea[key] += rouge_batch[key]

        # show batch result in file 
        for b in range(len(encoder_word_inputs_)):
            
            outf.write("{0}\t{1}\t{2}\n".format(ids2sent(test_x[b]), ids2sent(decoder_inputs_[b])
                                                , ids2sent(encoder_word_inputs_[b])))
            
    # compute rouge measue in the test
    outf.write("------rouge measure------")
    for key in rouge_mea.keys():
        rouge_mea[key] = rouge_mea[key] / (len(X_test)//batch_size)
        outf.write("{0}: {1}\n".format(key, rouge_mea[key]))

    
    outf.close()

In [ ]:
import numpy as np
sess = tf.Session()
sess.run(tf.global_variables_initializer())
graph = tf.get_default_graph()

if restore:
    saver.restore(sess, tf.train.latest_checkpoint(model_load_dir))
if inference:
    predict(X_test, Y_test, sess, inference_fn)

if train:
    print("start trainging...")  
    for i in range(epocs):
        j = 0
        while j < len(X_train):
            encoder_word_inputs_ = list(map(lambda x: rpadd(x, maxlend), X_train[j:j+batch_size]))
            encoder_pos_inputs_ = list(map(lambda x:rpadd(list(range(3,len(x)+3)), maxlend), X_train[j:j+batch_size]))
            decoder_inputs_ = list(map(lambda x: rpadd(x, maxlenh, prefix=beg), Y_train[j:j+batch_size]))
            decoder_targets_ = list(map(lambda x: x[1:]+[emp], decoder_inputs_))

            j = j + batch_size
            summary, _, loss_, decoder_prediction_ = sess.run([summary_op, train_op, loss, decoder_prediction],
                                                             feed_dict={
                                                                 encoder_word_inputs: encoder_word_inputs_,
                                                                 encoder_pos_inputs: encoder_pos_inputs_,
                                                                 decoder_inputs: decoder_inputs_,
                                                                 decoder_targets: decoder_targets_,
                                                                 feed_batch_size: len(encoder_word_inputs_),
                                                             })
            writer.add_summary(summary, i*len(X_train) + j)

            if j % (batch_size * 30) == 0:
                logger.info("Runing in epoc {0} batch {1} with loss {2}".format(i, j//batch_size, loss_))


        if i % save_epoc_step == 0:
            saver.save(sess, "{0}/TitleGeneration".format(model_dump_dir), global_step=i)
    print("finish trainging.") 



start trainging...


ResourceExhaustedError: OOM when allocating tensor with shape[128,20,378486]
	 [[Node: one_hot = OneHot[T=DT_FLOAT, TI=DT_INT32, axis=-1, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_decoder_targets_0_1/_13, one_hot/depth, one_hot/on_value, one_hot/off_value)]]
	 [[Node: decoder_prediction/_129 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7602_decoder_prediction", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'one_hot', defined at:
  File "/data1/songbohan/anaconda3/envs/reinforce_learning/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/data1/songbohan/anaconda3/envs/reinforce_learning/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/data1/songbohan/anaconda3/envs/reinforce_learning/lib/pytho

In [ ]:
from bleu import compute_bleu

with open("../log/test_beam.sample.v1.data", 'r', encoding="utf-8") as f:
    hypos = []
    refs = []
    hr, rr = [], []
    for line in f.readlines():
        hypo, ref, _ = line.strip().split("\t")
        hypos.append([token for token in hypo.split(" ")])
        refs.append([token for token in ref.split(" ")])
        hr.append(hypo)
        rr.append(ref)
    
#print(compute_bleu(refs, hypos))
print(rouge(hr, rr))

In [ ]:
# cite: https://arxiv.org/pdf/1702.01806.pdf
# title: Beam Search Strategies for Neural Machine Translation
# beam search with pruning (naive, version)
# prune hypo parameter

rp = 0.2
ap = 2.5
rpl = 0.3
mc = 2
beam_size = 30000

def prune(beam_tmp, decoder_probs_l):
    
#     print([[idx2word[idx] for idx in seq] for score, seq in beam_tmp])
    
#     max_score, max_seq = beam_tmp[0]
#     print("max_score: {0}, max_seq: {1}".format(max_score, max_seq))
#     beam_tmp = list(filter(lambda x: x[0] > rp * max_score, beam_tmp))
#     beam_tmp = list(filter(lambda x: x[0] > max_score - ap, beam_tmp))
#     beam_tmp = list(filter(lambda x: decoder_probs_l[x[1][-1]] > rpl * decoder_probs_l[max_seq[-1]], beam_tmp))
    
#     beam_res = []
#     cnt = {}
#     for prob, seq in beam_tmp:
#         key = "_".join([str(s) for s in seq])
#         if key not in cnt:
#             cnt[key] = 0
#         if cnt[key] < mc:
#             beam_res.append((prob, seq))
#             cnt[key] += 1
            
#     print([[idx2word[idx] for idx in seq] for score, seq in beam_res])

    
    
    return beam_res

def beam_search_with_pruning(X_one, Y_one, sess, beam_size):
    
    encoder_word_inputs_ = [rpadd(X_one, maxlend)]
    encoder_pos_inputs_ = [rpadd(list(range(3,len(X_one)+3)), maxlend)]
    
    beam_list = [(1.0, [beg])] 
    final_list = []
    
    for l in range(maxlenh):

        candidate = {}
        for prob_seq, seq in beam_list:
            
            
            new_decoder_input_ = [rpadd(seq, maxlenh, prefix=None)]
   
            decoder_logits_ = sess.run(tf.log(tf.nn.softmax(decoder_logits)),
                                          feed_dict={
                                              encoder_word_inputs: encoder_word_inputs_,
                                              encoder_pos_inputs: encoder_pos_inputs_,
                                              decoder_inputs: new_decoder_input_,
                                              feed_batch_size: 1
                                          })
            decoder_probs_l = decoder_logits_[0][l]
            probs_top_ids = np.argsort(decoder_probs_l)[-beam_size:]
#             print([idx2word[idx] for idx in probs_top_ids])
            
#             print("------------------------------")
            
            for v in probs_top_ids:
                pt = prob_seq + decoder_probs_l[v]
                candidate[pt] = seq + [v]
                
       
 
        res_sort = sorted(list(candidate.items()), key=lambda x: x[0], reverse=True)   
        
        beam_tmp = res_sort[:beam_size]
        
        # beam_tmp = prune(beam_tmp, decoder_probs_l)  
                
        beam_list.clear()
        
        for prob, seq in beam_tmp:
            if seq[-1] == eos:
                final_list.append((prob, seq))
                beam_size -= 1
            else:
                beam_list.append((prob, seq))
            
        if len(beam_list) == 0 or beam_size == 0:
            break
           
    return [(ids2sent(item[1]), item[0]) for item in final_list]

In [ ]:
def predict_with_beam(X_test, Y_test, sess, beam_size):
    
    out = open("./beam_test", "w", encoding="utf-8")
    for x, y in zip(X_test, Y_test):
        res_list = beam_search_with_pruning(x, y, sess, 10)
        out.write("{0}\t{1}\t{2}\n".format(res_list[0], ids2sent(y), ids2sent(x)))
        out.flush()
    out.close()


In [ ]:
ner_set = set(["DATE",
            "IDEOLOGY",
            "CITY",
            "MISC",
            "STATE_OR_PROVINCE",
            "DEMONYM",
            "TIME",
            "ORGANIZATION",
            "TITLE",
            "MONEY",
            "CRIMINAL_CHARGE",
            "COUNTRY",
            "PERSON",
            "CAUSE_OF_DEATH",
            "FACILITY",
            "PERCENT",
            "ORDINAL",
            "NUMBER",
            "LOCATION",
            "RELIGION",
            "NATIONALITY",
            "GPE"])


test_content_words = []
test_content_ner2words = []
with open("../data/test_content_words", "r", encoding="utf-8") as wf:
    with open("../data/test_content_ners", "r", encoding="utf-8") as nf:
        
        for wl, nl in zip(wf.readlines(), nf.readlines()):
            
            _, words = wl.strip().split("\t")
            _, ners = nl.strip().split("\t")
            ner2words = {}
            for word, ner in zip(words.split("\3"), ners.split("\3")):
                ner = "<"+ner+">"
                if ner == '<O>': continue
                if ner not in ner2words:
                    ner2words[ner] = []
                ner2words[ner].append(word)
            
            test_content_ner2words.append(ner2words)
            test_content_words.append(" ".join(words.split("\3")))
            


In [ ]:
ner_set = set(["DATE",
            "IDEOLOGY",
            "CITY",
            "MISC",
            "STATE_OR_PROVINCE",
            "DEMONYM",
            "TIME",
            "ORGANIZATION",
            "TITLE",
            "MONEY",
            "CRIMINAL_CHARGE",
            "COUNTRY",
            "PERSON",
            "CAUSE_OF_DEATH",
            "FACILITY",
            "PERCENT",
            "ORDINAL",
            "NUMBER",
            "LOCATION",
            "RELIGION",
            "NATIONALITY",
            "GPE"])


test_content_words = []
test_content_ner2words = []
test_content_word_ner = []
with open("../data/test_content_words", "r", encoding="utf-8") as wf:
    with open("../data/test_content_ners", "r", encoding="utf-8") as nf:
    
        for wl, nl in zip(wf.readlines(), nf.readlines()):
            
            _, words = wl.strip().split("\t")
            _, ners = nl.strip().split("\t")
            ner2words = {}
            tmp = []
            for word, ner in zip(words.split("\3"), ners.split("\3")):
                ner = "<"+ner+">"
                if ner == '<O>': continue
                if ner not in ner2words:
                    ner2words[ner] = []
                ner2words[ner].append(word)
                tmp.append([word,ner])
            test_content_word_ner.append(tmp)
            test_content_ner2words.append(ner2words)
            test_content_words.append(" ".join(words.split("\3")))
            

            
def find_adapt_word(token, pre_word, next_word, ori_text):
    
    print(token, pre_word, next_word, ori_text)
    
    for idx, (word, ner) in enumerate(ori_text):
        if word == pre_word and ori_text[idx+1][1]==token:
            return ori_text[idx+1][0]
        if word == next_word and ori_text[idx-1][1]==token:
            return ori_text[idx-1][0]
        
    return None
            
def predict_online(X_test, Y_test, sess, test_content_words):
    
    N = len(X_test)
    beam_size = 10
    outf = open("../data/produce.beam.test1", "w", encoding="utf-8")
    for idx in range(N):
        x, y = X_test[idx], Y_test[idx]
        candiate = beam_search_with_pruning(x, y, sess, beam_size)
        can = find_proper(candiate, test_content_words[idx], 5)
        print(can)
        filled = []
        words = can.split(" ")[1:]
        for udx, word in enumerate(words):
            if word in test_content_ner2words[idx]:
#                 ada_word = find_adapt_word(word, words[udx-1], words[udx+1], test_content_word_ner[idx])
                ada_word = None
                if ada_word:
                    filled.append(ada_word)
                else:
                    filled.append(test_content_ner2words[idx][word][0])
            else:
                filled.append(word)
        outf.write("{0}\t{1}\t{2}\n".format("".join(filled),ids2sent(y), "".join(test_content_words[idx].split(" "))))
        outf.flush()
    outf.close()

In [ ]:

def remove_same_word(text):
    tmp = set()
    new_text = []
    for t in text:
        if t in tmp: continue
        new_text.append(t)
        tmp.add(t)
        
    return new_text


def find_proper(t, ground_truth, top):

#     minn = -1e9
#     text = None
#     for a in t:
#         s1 = " ".join(remove_same_word([b.lower() for b in a[0].split(" ") if b not in set("。，；““！？了")]))
#         s2 = " ".join([w.lower() for w in ground_truth.split(" ") if w not in set("。，；““！？了")])
#     #     print(s1)
#     #     print(s2)
#         score = rouge(s1,s2)["rouge_2/r_score"]*0.8+rouge(s1,s2)["rouge_1/r_score"]*0.2
#         print(a)
#         print(score)
#         print(rouge(s1,s2)["rouge_2/r_score"])
#         if score > minn:
#             minn = score
#             text = a[0]
            
    tmp = ([(rouge(" ".join(remove_same_word([b.lower() for b in a[0].split(" ") if b not in set("。，；““！？了")])),
              " ".join(remove_same_word([w.lower() for w in ground_truth.split(" ") if w not in set("。，；““！？了")]))),a[0])
             for a in t])
    def score(x):
        return x["rouge_2/r_score"]*0.8+x["rouge_l/r_score"]*0.2
    tmp = sorted(tmp, key=lambda x:score(x[0]), reverse=True)[:top]
    s =  min(tmp, key=lambda x:len(x[1].split(" ")))[1]

    return " ".join(remove_same_word(s.split(" ")))



In [ ]:
predict_online(X_test, Y_test, sess, test_content_words)

In [ ]:
t = beam_search_with_pruning(X_test[3], Y_test[3], sess, 10)

In [ ]:
find_proper(t, test_content_words[3], 5)

In [ ]:
" ".join([w for w in test_content_words[0].split(" ") if w not in stopwords])

In [ ]:
t

In [ ]:
real = []
for word in can.split(" ")[1:]:
    if word in test_content_ner2words[1]:
        real.append(test_content_ner2words[1][word][0])
    else:
        real.append(word)
print(real)

In [ ]:
test_content_ner2words[3]

In [ ]:
for a in t:
    r = rouge([a[0]], [test_content_words[3]])
    print(a[0])
    print(test_content_words[3])
    print(r)

In [ ]:
import scipy
1 - scipy.spatial.distance.cosine(embed[word2idx["你"]],embed[word2idx["我"]])

In [ ]:
# 测试词向量

def neighbor(target_word, k):
    
    w2sim = {}
    for word in word2idx.keys():
        sim = 1 - scipy.spatial.distance.cosine(embed[word2idx[target_word]],embed[word2idx[word]]) 
        w2sim[word] = sim
    
    return sorted(w2sim.items(), key=lambda x:x[1], reverse=True)[:k]

In [ ]:
ns = neighbor("外汇", 20)
for n in ns:
    print(n)